In [1]:
from engine import Engine

In [3]:
# using the built in run loop
engine = Engine(caption='Built in run loop')

def callback():
    pass

def mouse_pressed():
    print('mouse: {}'.format(engine.mouse))

engine.run(callback=callback, mouse_pressed=mouse_pressed)

Running...
mouse: (344, 348)
mouse: (296, 227)
mouse: (218, 173)


In [4]:
# using a custom run loop

engine = Engine(caption='Custom run loop')

running = True
while running:
    for e in engine.events():
        if engine.quit_event(e):
            running = False
            
    engine.step_physics()
    engine.render()
    engine.clock_tick()
    
engine.close()

In [17]:
# using a custom run loop
import util
import numpy as np
from engine import Engine

sig = lambda x: 1.0/(1.0+np.exp(-x))

# sparse:
sparsity = 30
def act(x):
    x = x.copy()
    idx = np.argpartition(x, sparsity)[0, :sparsity]
    x[0, idx] = 0
    return x

def act_p(x):
    idx = np.argpartition(x, sparsity)[0, :sparsity]
    mask = np.ones_like(x)
    mask[0, idx] = 0
    return mask

W = np.random.randn(32, 32)
h = np.random.randn(1, 32)

gW = np.zeros_like(W)

engine = Engine(caption='Custom run loop', width=1280, height=720)

seq = 0

running = True
while running:
    for e in engine.events():
        if engine.quit_event(e):
            running = False
    
    
    h1 = np.clip(act(h.dot(W)), -1, 1)
    
    gW += np.multiply(h.T, h1)
    gW = np.clip(gW, -1, 1)
    
    if h1[0, seq] > 0:
        W += 0.01 * gW
        
    W = np.clip(W, -1, 1)
    W *= 0.9998
    W += np.random.randn(W.shape[0], W.shape[1]) * 0.01
    gW *= 0.9
    
    
    targ = np.zeros_like(h)
    targ[0, seq] = 1
    
    
    util.render_matrices(engine.screen, [h.T, W, h1.T, targ.T, gW])
    
    seq += 1
    if seq >= h1.shape[1]:
        seq = 0
            
    h = h1.copy() + np.random.randn(h1.shape[0], h1.shape[1]) * 0.1
    
    engine.render()
    engine.clock_tick()
    
engine.close()